In [43]:
import pandas as pd
import os
import re
import warnings
from decimal import Decimal#保证比例呈现两位小数
import sys
import numpy as np
from datetime import datetime
from docx import Document
from docx.enum.style import WD_STYLE_TYPE
import pymssql
from docx.enum.text import WD_ALIGN_PARAGRAPH  #设置对象居中、对齐等。
from docx.enum.text import WD_TAB_ALIGNMENT,WD_TAB_LEADER,WD_LINE_SPACING  #设置制表符等
from docx.shared import Inches   #设置图像大小
from docx.shared import Pt,Cm   #设置像素、缩进等
from docx.shared import RGBColor    #设置字体颜色
from docx.shared import Length    #设置宽度
from docx.oxml.ns import qn  #设置中文版式
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL #垂直居中
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from docx.table import _Cell
from docx.oxml import OxmlElement
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
import functools
import keyboard
warnings.filterwarnings('ignore')

# 读取excel

In [2]:
try:
    excel_path = '数据'
    model_path = '模板'
    file_names = os.listdir(excel_path)
    file_names = [file for file in file_names if file.endswith(('.xlsx','.xls','.XLSX','.XLS'))]

    data_group=[]
    for file in file_names:
        file_path = os.path.join(excel_path, file)  # 构建文件的完整路径
        df = pd.read_excel(file_path)
        data_group.append(df)

    key_group={}
    for index in range(len(data_group)):
        key_group[data_group[index].columns[0]]=index
        
    print('程序读取excel成功！')
except:
    print('程序读取excel报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 自定义函数

In [5]:
# 取data的前三，可自动忽略汇总行和总计行，忽略非数字,不忽略百分数
def get_3rd(data,num,not_3=3):#num为倒序排序的依据,不取前三也行
    datat=data.copy()
    datat= datat[(datat.iloc[:,0] != '汇总') & (datat.iloc[:,0] != '总计')& (datat.iloc[:,0] != 'left') & (datat.iloc[:,1] != '汇总') & (datat.iloc[:,1] != '总计')]
    datat['temp']=data.apply(lambda x:(float(re.sub('%','',x[num])) if '%' in x[num] else 0) if isinstance(x[num],str) else x[num],axis=1)
    datat=datat.sort_values(by='temp',ascending=False).reset_index(drop=True).drop(['temp'],axis=1)
    return [datat.loc[x].tolist() for x in range(datat.shape[0]>not_3 and not_3 or datat.shape[0])]

In [6]:
def mssql_data(sql,lua=[]):
    conn = pymssql.connect(server='39.104.209.155', user='sa', password='admin_0', database='hn_szx')
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()
    data=pd.DataFrame(data, columns=[column[0] for column in cursor.description])
    for i in lua:
        data[i]=data[i].apply(lambda x:x.encode('latin1').decode('gbk'))
    cursor.close()
    conn.close()
    return data

In [7]:
def add_p(content1,content2):
    global document
    document.add_paragraph(content1, style='CustomStyle2')
    document.add_paragraph(content2, style='CustomStyle1')

In [8]:
def xy(datat,x,x1,y):
    return datat[datat[x]==x1][y].values[0]

def change_to_decimal(x):
    return  str(Decimal(x*100).quantize(Decimal("0.00")))+'%'


In [9]:
def Set_Background_Color(cell,rgbColor):
    shading_elm = parse_xml(r'<w:shd {} w:fill="{color_value}"/>'.format(nsdecls('w'),color_value = rgbColor)) #固定写法，照抄即可
    cell._tc.get_or_add_tcPr().append(shading_elm)
    
def add_table(datat,tail=[]):#自动加首行
    data=datat.copy()
    cols=data.columns
    #总计行
    if len(tail)==data.shape[1]:
        dict2={cols[i]:[tail[i]] for i in range(len(cols))}
        datatt=pd.DataFrame(dict2)
        data=pd.concat([data,datatt],ignore_index=True)
    global document
    row_count = data.shape[0]
    col_count = data.shape[1]
    table = document.add_table(rows=row_count+1, cols=col_count,style='table1')#table1只是保证有边框
        
    table.autofit = True #自动调整宽度和高度
    
    # 单元格样式：居中对齐，边框颜色为黑色，仿宋_GB2312小四字体
    cell_format = table.style.paragraph_format
    cell_format.alignment = WD_ALIGN_VERTICAL.CENTER   # 垂直居中对齐
    
    # 填充数据
    for i, column_name in enumerate(data.columns):
        cell = table.cell(0, i)
        cell.text = column_name
        cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # 设置标题四号黑体字体
        cell.paragraphs[0].runs[0].font.size = Pt(14)
        cell.paragraphs[0].runs[0].bold = True
        cell.paragraphs[0].runs[0].font.name = '黑体'
        cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '黑体')#中文要这样设置
        Set_Background_Color(cell,RGBColor(220,230,241))
        for j in range(1, data.shape[0] + 1):
            cell = table.cell(j, i)
            cell.text = str(data.iloc[j - 1, i])

            # 设置仿宋_GB2312小四字体
            cell.paragraphs[0].runs[0].bold = False
            cell.paragraphs[0].runs[0].font.name = '仿宋_GB2312'
            cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '仿宋_GB2312')#中文要这样设置
            cell.paragraphs[0].runs[0].font.size = Pt(12)
            if i!=0:
                cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    # 处理最后一行
    if tail:
        last_row = table.rows[-1]
        # 遍历最后一行的单元格
        for cell in last_row.cells:
            if cell.text=='总计':
            # 获取单元格的段落
                cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                paragraph = cell.paragraphs[0]
                cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '黑体')#中文要这样设置
                cell.paragraphs[0].runs[0].font.size = Pt(14)
                cell.paragraphs[0].runs[0].bold = True
            Set_Background_Color(cell,RGBColor(220,230,241))
    document.add_paragraph().add_run('')#增加空行

# 数据处理

In [10]:
try:
    current_time = datetime.now()

    year = current_time.year
    month = current_time.month
    day = current_time.day
    graduate_year = month >8 and year+1 or year

    title='各性别就业率'
    data=data_group[key_group[title]]

    bysrs=int(xy(data,title,'合计','毕业人数小计'))
    qxlsl=change_to_decimal(xy(data,title,'合计','就业率小计'))
    boy_qxlsl=change_to_decimal(xy(data,title,'1男','就业率小计'))
    girl_qxlsl=change_to_decimal(xy(data,title,'2女','就业率小计'))

    zk_bysrs=int(xy(data,title,'合计','专科毕业人数'))
    zk_jyrs=int(xy(data,title,'合计','专科就业人数'))
    zk_qxlsl=change_to_decimal(xy(data,title,'合计','专科就业率'))
    zk_boy_qxlsl=change_to_decimal(xy(data,title,'1男','专科就业率'))
    zk_girl_qxlsl=change_to_decimal(xy(data,title,'2女','专科就业率'))
    bk_bysrs=int(xy(data,title,'合计','本科毕业人数'))
    bk_jyrs=int(xy(data,title,'合计','本科就业人数'))
    bk_qxlsl=change_to_decimal(xy(data,title,'合计','本科就业率'))
    bk_boy_qxlsl=change_to_decimal(xy(data,title,'1男','本科就业率'))
    bk_girl_qxlsl=change_to_decimal(xy(data,title,'2女','本科就业率'))
    ss_bysrs=int(xy(data,title,'合计','硕士生毕业人数'))
    ss_jyrs=int(xy(data,title,'合计','硕士生就业人数'))
    ss_qxlsl=change_to_decimal(xy(data,title,'合计','硕士生就业率'))
    ss_boy_qxlsl=change_to_decimal(xy(data,title,'1男','硕士生就业率'))
    ss_girl_qxlsl=change_to_decimal(xy(data,title,'2女','硕士生就业率'))
    bs_bysrs=int(xy(data,title,'合计','博士生毕业人数'))
    bs_jyrs=int(xy(data,title,'合计','博士生就业人数'))
    bs_qxlsl=change_to_decimal(xy(data,title,'合计','博士生就业率'))
    bs_boy_qxlsl=change_to_decimal(xy(data,title,'1男','博士生就业率'))
    bs_girl_qxlsl=change_to_decimal(xy(data,title,'2女','博士生就业率'))

    title='各生源地（省）就业率'
    data=data_group[key_group[title]]
    data.head()

    zk_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','专科就业率'))
    bk_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','本科就业率'))
    ss_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','硕士生就业率'))
    bs_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','博士生就业率'))
    hn_jybl=change_to_decimal(xy(data,title,'43湖南省','就业率小计'))

    title='就业地区东中西分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    jydq1_name=data.loc[1][title]
    jydq2_name=data.loc[2][title]
    jydq3_name=data.loc[3][title]
    jydq_bl1=change_to_decimal(data.loc[1][bl_name])
    jydq_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业地区省份分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    jydqsf1_name=re.sub('\d','',data.loc[1][title])
    jydqsf2_name=re.sub('\d','',data.loc[2][title])
    jydqsf_bl1=change_to_decimal(data.loc[1][bl_name])
    jydqsf_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业单位性质分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)

    jydwxz1_name=re.sub('\d','',data.loc[1][title])
    jydwxz2_name=re.sub('\d','',data.loc[2][title])
    jydwxz3_name=re.sub('\d','',data.loc[3][title])
    jydwxz4_name=re.sub('\d','',data.loc[4][title])
    jydwxz_bl1=change_to_decimal(data.loc[1][bl_name])
    jydwxz_bl2=change_to_decimal(data.loc[2][bl_name])

    title='职位类型分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    # display(data)
    jyzwlx1_name=re.sub('\d','',data.loc[1][title])
    jyzwlx2_name=re.sub('\d','',data.loc[2][title])
    jyzwlx_bl1=change_to_decimal(data.loc[1][bl_name])
    jyzwlx_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业单位行业分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    # display(data)
    jydwhy1_name=re.sub('\d','',data.loc[1][title])
    jydwhy2_name=re.sub('\d','',data.loc[2][title])
    jydwhy_bl1=change_to_decimal(data.loc[1][bl_name])
    jydwhy_bl2=change_to_decimal(data.loc[2][bl_name])
    print('程序数据处理成功')
except:
    print('程序数据处理报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 替换word模板

In [21]:
try:
    document=Document(model_path+'/模板.docx')
    replace_name=['year','month','day',
    'bysrs',
     'qxlsl',
     'boy_qxlsl',
     'girl_qxlsl',
     'hn_jybl',
     'zk_bysrs',
     'zk_jyrs',
     'zk_qxlsl',
     'zk_boy_qxlsl',
     'zk_boy_qxlsl',
     'zk_hn_jybl',
     'bk_bysrs',
     'bk_jyrs',
     'bk_qxlsl',
     'bk_boy_qxlsl',
     'bk_boy_qxlsl',
     'bk_hn_jybl',
     'ss_bysrs',
     'ss_jyrs',
     'ss_qxlsl',
     'ss_boy_qxlsl',
     'ss_boy_qxlsl',
     'ss_hn_jybl',
     'bs_bysrs',
     'bs_jyrs',
     'bs_qxlsl',
     'bs_boy_qxlsl',
     'bs_boy_qxlsl',
     'bs_hn_jybl',
     'jydq1_name',
     'jydq_bl1',
     'jydq2_name',
     'jydq_bl2',
     'jydq3_name',
     'jydqsf1_name',
     'jydqsf_bl1',
     'jydqsf2_name',
     'jydqsf_bl2',
     'jydwxz1_name',
     'jydwxz_bl1',
     'jydwxz2_name',
     'jydwxz_bl2',
     'jydwxz3_name',
     'jydwxz4_name',
     'jyzwlx1_name',
     'jyzwlx_bl1',
     'jyzwlx2_name',
     'jyzwlx_bl2',
     'jydwhy1_name',
     'jydwhy_bl1',
     'jydwhy2_name',
     'jydwhy_bl2']

    for j in range(len(document.paragraphs)):
        flag=False
        if len(document.paragraphs[j].runs)>0:#先获取run的格式，不然后面replace似乎就没有格式了
            fname=document.paragraphs[j].runs[0].font.name
            fsize=document.paragraphs[j].runs[0].font.size
        for i in replace_name:
            if '【'+i+'】' in document.paragraphs[j].text:
                flag=True
                document.paragraphs[j].text = document.paragraphs[j].text.replace('【'+i+'】',str(eval(i)))
        if flag:
            for run in document.paragraphs[j].runs:
                run.font.name=fname
                run._element.rPr.rFonts.set(qn('w:eastAsia'), fname)
                run.font.size=fsize
    print('程序替换word模板成功！')
except:
    print('程序替换word模板报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 八、 访企拓岗总体情况

In [23]:
try:
    title='username'
    data=data_group[key_group[title]][['qa3_1','qa3_2','qa5_1']]
    data['qa3_1']=data['qa3_1'].apply(lambda x:x if x>0 else 0)
    data['qa3_2']=data['qa3_2'].apply(lambda x:x if x>0 else 0)
    data['qa5_1']=data['qa5_1'].apply(lambda x:x if x>0 else 0)
    data['temp']=data['qa5_1'].apply(lambda x:1 if x>0 else 0)
    xzjygw=int(sum(data['qa3_1'])+sum(data['qa3_2']))
    sxsjjd=int(sum(data['temp']))
    xjnsxs=int(sum(data['qa5_1']))

    content1=f"\t 八、 访企拓岗总体情况"
    content2=f"截至{year}年{month}月{day}日9点整，为促进20xx届毕业生就业，全省共有xxx所高校开展访企拓岗活动，已走访xxx家单位，促成企业新增就业岗位{xzjygw}个，新签订{sxsjjd}家实习实践基地、拟新接纳实习生{xjnsxs}人。"
    add_p(content1,content2)
    
    print('访企拓岗导入成功！')
except:
    print('访企拓岗报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 九、 校招情况

In [25]:
try:
    sql=f'''
    select *,FORMAT(meet_time, 'yyyy年M月') as f_time from (
    SELECT
        b.Name AS school_name,
        SchoolId AS school_id,
        CAST(a.Id AS VARCHAR) AS career_talk_id,
        Title AS title,
        MeetDate AS meet_time,
        CASE
            WHEN MONTH(MeetDate) > 8 THEN YEAR(MeetDate) + 1
            WHEN MONTH(MeetDate) < 9 THEN YEAR(MeetDate)
            ELSE NULL
        END AS graduate_year,
        CASE
            WHEN MONTH(MeetDate) > 8 OR MONTH(MeetDate) < 3 THEN '上半年'
            WHEN MONTH(MeetDate) < 9 AND MONTH(MeetDate) > 2 THEN '下半年'
            ELSE NULL
        END AS [学期],
        CASE
            WHEN RecruitType = 0 THEN '正式宣讲'
            ELSE '实习宣讲'
        END AS type_desc
    FROM
        bus_career a
    LEFT JOIN
        sys_org b ON a.SchoolId = b.Id
    WHERE
        (
            CASE
                WHEN MONTH(MeetDate) > 8 THEN YEAR(MeetDate) + 1
                WHEN MONTH(MeetDate) < 9 THEN YEAR(MeetDate)
                ELSE NULL
            END
            BETWEEN YEAR(GETDATE()) - 3 AND YEAR(GETDATE()) + 1
        )
        AND IsPublish = 1
        AND State <> 1
        AND a.IsDeleted = 0
        AND SchoolId IN (select distinct Id from
    (select DISTINCT c.Id,c.Name
    from sys_api_caller_log a
    inner join bus_caller b ON a.BusCallerId = b.Id
    inner join sys_org c ON b.SysOrgId = c.Id
    WHERE c.IsDeleted = 0 AND a.IsDeleted = 0 AND b.IsDeleted = 0
    UNION
    select '353373162956734','长沙医学院') a)
        AND b.IsDeleted = 0
        )a 
        where graduate_year={graduate_year};'''

    data_xjh=mssql_data(sql,['学期','type_desc'])
    data_xjh

    sql=f'''select *,FORMAT(meet_time, 'yyyy年M月') as f_time from(
    SELECT
        b.Name AS school_name,
        a.SchoolId AS school_id,
        CAST(a.Id AS VARCHAR) AS fair_id,
        a.Title AS title,
        MeetTime AS meet_time,
        CASE
            WHEN MONTH(MeetTime) > 8 THEN YEAR(MeetTime) + 1
            WHEN MONTH(MeetTime) < 9 THEN YEAR(MeetTime)
            ELSE NULL
        END AS graduate_year,
        CASE
            WHEN MONTH(MeetTime) > 8 OR MONTH(MeetTime) < 3 THEN '上半年'
            WHEN MONTH(MeetTime) < 9 AND MONTH(MeetTime) > 2 THEN '下半年'
            ELSE NULL
        END AS [学期],
        CASE
            WHEN a.Type = '0' THEN '线上双选会'
            WHEN a.Type = '1' THEN '线下双选会'
            ELSE NULL
        END AS type_desc
    FROM
        bus_bothmeet a
    LEFT JOIN
        sys_org b ON a.SchoolId = b.Id
    WHERE
        a.IsDeleted = 0
        AND (a.XxAuditStatus = '2' OR ISNULL(a.XxAuditStatus, '') = '')
        AND b.IsDeleted = 0
        AND (
            CASE
                WHEN MONTH(MeetTime) > 8 THEN YEAR(MeetTime) + 1
                WHEN MONTH(MeetTime) < 9 THEN YEAR(MeetTime)
                ELSE NULL
            END
            BETWEEN YEAR(GETDATE()) - 3 AND YEAR(GETDATE()) + 1
        )
        AND b.Id IN (select distinct Id from
    (select DISTINCT c.Id,c.Name
    from sys_api_caller_log a
    inner join bus_caller b ON a.BusCallerId = b.Id
    inner join sys_org c ON b.SysOrgId = c.Id
    WHERE c.IsDeleted = 0 AND a.IsDeleted = 0 AND b.IsDeleted = 0
    UNION
    select '353373162956734','长沙医学院') a)
    )a
    where graduate_year={graduate_year}
    '''

    data_sxh=mssql_data(sql,['学期','type_desc'])
    data_sxh

    sql=f'''
    select * from(
    SELECT 
      b.SchoolName AS school_name,
      b.SchoolId AS school_id,
      '湖南省' AS xxsf,
      CAST(a.BothMeetId AS VARCHAR) AS fair_id,
      b.Title AS title,
      b.MeetTime AS meet_time,
      a.CompanyName AS company_name,
      ApplyTime AS approve_time,
      CASE 
        WHEN MONTH(b.MeetTime) > 8 THEN YEAR(b.MeetTime) + 1
        WHEN MONTH(b.MeetTime) < 9 THEN YEAR(b.MeetTime)
        ELSE NULL 
      END AS graduate_year,
      CASE 
        WHEN b.Type = '0' THEN '双选会'
        WHEN b.Type = '1' THEN '专场招聘会'
        ELSE NULL 
      END AS type_desc
    FROM 
      bus_bothmeet_company a
    LEFT JOIN 
      bus_bothmeet b ON a.BothMeetId = b.Id
    LEFT JOIN 
      sys_org c ON b.SchoolId = c.Id
    WHERE 
      b.IsDeleted = 0 
      AND a.AuditStatus != 2 
      AND YEAR(b.MeetTime) BETWEEN YEAR(GETDATE()) - 3 AND YEAR(GETDATE()) + 1 
      AND c.IsDeleted = 0)a
      where graduate_year={graduate_year}'''

    sxh_qy=mssql_data(sql,['xxsf','type_desc'])
    sxh_qy

    sql=f'''
    select * from(
    SELECT
    CONVERT(VARCHAR(10), PublishTime, 120) AS publish_time,
    '' AS fair_id,
    b.CompanyName AS company_name,
    CASE
    WHEN MONTH(PublishTime) > 8 THEN YEAR(PublishTime) + 1
    WHEN MONTH(PublishTime) < 9 THEN YEAR(PublishTime)
    ELSE NULL
    END AS graduate_year,
    CASE
    WHEN MONTH(PublishTime) > 8 OR MONTH(PublishTime) < 3 THEN '上半年'
    WHEN MONTH(PublishTime) < 9 AND MONTH(PublishTime) > 2 THEN '下半年'
    ELSE NULL
    END AS 学期,
    '' AS school_name,
    '' AS school_id,
    ISNULL(COUNT(b.JobNumber), 0) AS job_num,
    ISNULL(SUM(CAST(b.JobNumber AS BIGINT)), 0) AS num
    FROM bus_company_job b
    WHERE b.JobStatus = 1 AND b.IsDeleted = 0 AND b.JobNumber <= 200
    GROUP BY
    CONVERT(VARCHAR(10), PublishTime, 120),
    CASE
    WHEN MONTH(PublishTime) > 8 THEN YEAR(PublishTime) + 1
    WHEN MONTH(PublishTime) < 9 THEN YEAR(PublishTime)
    ELSE NULL
    END,
    CASE
    WHEN MONTH(PublishTime) > 8 OR MONTH(PublishTime) < 3 THEN '上半年'
    WHEN MONTH(PublishTime) < 9 AND MONTH(PublishTime) > 2 THEN '下半年'
    ELSE NULL
    END,
    b.CompanyName)a
    where graduate_year={graduate_year} '''

    qyzpxq=mssql_data(sql)#企业招聘详情
    qyzpxq

    sql=f'''
    select sum(num) as num from (
    select count(Id) as  num from bus_career_student 
    where CASE
            WHEN MONTH(CreatedTime) > 8 THEN YEAR(CreatedTime) + 1
            WHEN MONTH(CreatedTime) < 9 THEN YEAR(CreatedTime)
            ELSE NULL
        END={graduate_year}
    union
    select count(Id) as  num from bus_bothmeet_student 
    where CASE
            WHEN MONTH(CreatedTime) > 8 THEN YEAR(CreatedTime) + 1
            WHEN MONTH(CreatedTime) < 9 THEN YEAR(CreatedTime)
            ELSE NULL
        END={graduate_year})a'''
    chxs_num=mssql_data(sql)#参会学生人数

    sql=f'''select distinct Id from
    (select DISTINCT c.Id,c.Name
    from sys_api_caller_log a
    inner join bus_caller b ON a.BusCallerId = b.Id
    inner join sys_org c ON b.SysOrgId = c.Id
    WHERE c.IsDeleted = 0 AND a.IsDeleted = 0 AND b.IsDeleted = 0
    UNION
    select '353373162956734','长沙医学院') a'''
    jcgxmd=mssql_data(sql)#监测高校名单

    sql=f'''SELECT count(a.Id)
    FROM bus_company a
    LEFT JOIN sys_dict_data b ON a.IndustryCode = b.Value
    WHERE a.IsDeleted=0 AND AuditStatus IN (1,2,3) AND b.IsDeleted = 0 AND b.DictTypeId = '269037954100016';
    '''
    zkdws=mssql_data(sql)#在库单位数

    sql=f'''select count(Id) as  num from bus_job_resume_relation 
    where CASE
            WHEN MONTH(SendTime) > 8 THEN YEAR(SendTime) + 1
            WHEN MONTH(SendTime) < 9 THEN YEAR(SendTime)
            ELSE NULL
        END={graduate_year}'''
    jltds=mssql_data(sql)#简历投递数

    content1=f"\t 九、 校招情况"
    content2=f'''全省部分高校（{jcgxmd.shape[0]}所）面向{graduate_year}届毕业生开展宣讲会{data_xjh.shape[0]}场，双选会{data_sxh.shape[0]}场，高校用人单位库{zkdws.shape[0]}家，2023届活跃单位数量{len(qyzpxq['company_name'].unique())}家，提供岗位{sum(qyzpxq['job_num'])}个，投递简历{jltds.shape[0]}个。'''

    add_p(content1,content2)
    
    print('校招情况成功！')
except:
    print('校招情况报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

,school_name,school_id,career_talk_id,title,meet_time,graduate_year,学期,type_desc,f_time
0,湖南大学,353373162938401,430468075807860,2023年江苏省扬州市邗江区优秀青年人才选聘,2023-06-19 09:00:00,2023,下半年,正式宣讲,2023年6月
1,湖南大学,353373162938401,430468075808364,实习生招聘,2023-06-20 09:00:00,2023,下半年,实习宣讲,2023年6月


# 十、 分月趋势

In [34]:
try:
    data1=data_sxh.query(f"type_desc=='线上双选会'").groupby('f_time').size().reset_index()
    data1.columns=['日期','视频双选会']
    data2=data_sxh.query(f"type_desc=='线下双选会'").groupby('f_time').size().reset_index()
    data2.columns=['日期','双选会']
    data3=data_xjh.groupby('f_time').size().reset_index()
    data3.columns=['日期','宣讲会']
    datat=pd.merge(data1,data2,on='日期',how='outer')
    datat=pd.merge(datat,data3,on='日期',how='outer')
    datat=datat.fillna(0)
    datat['总计']=datat.apply(lambda x:x['视频双选会']+x['双选会']+x['宣讲会'],axis=1)
    datat=datat.applymap(lambda x:int(x) if isinstance(x,float) else x)
    datat['order']=datat['日期'].apply(lambda x:re.sub('9','09',x))
    datat=datat.sort_values(by='order')
    datat.drop('order',axis=1,inplace=True)

    content1=f"\t 十、 分月趋势"
    content2=f'''分月校园招聘活动开展情况如下表所示：'''
    add_p(content1,content2)
    add_table(datat,tail=['总计',sum(datat['视频双选会']),sum(datat['双选会']),sum(datat['宣讲会']),sum(datat['总计'])])
    
    print('分月趋势成功！')
except:
    print('分月趋势报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 十一、 分行业需求

In [36]:
try:
    sql=f'''SELECT case when a.CompanyIndustry in ('信息传输、软件和信息技术服务业','计算机/互联网/通信/电子','软件和信息技术服务业') then '信息传输、软件和信息技术服务业'
             when a.CompanyIndustry in ('建筑业','房地产/建筑') then '建筑业'
             when a.CompanyIndustry in ('科学研究和技术服务业') then '科学研究和技术服务业'
             when a.CompanyIndustry in ('租赁和商务服务业') then '租赁和商务服务业'
             when a.CompanyIndustry in ('批发和零售业','批发业','零售业') then '批发和零售业'
             when a.CompanyIndustry in ('教育','教育业','专业服务/教育/培训') then '教育'
             when a.CompanyIndustry in ('金融业','会计/金融/银行/保险') then '金融业'
             when a.CompanyIndustry in ('制造业','贸易/消费/制造/营运') then '制造业'
             when a.CompanyIndustry in ('住宿和餐饮业') then '住宿和餐饮业'
             when a.CompanyIndustry in ('公共管理、社会保障和社会组织','政府/非盈利机构/其他') then '公共管理、社会保障和社会组织'
             when a.CompanyIndustry in ('卫生和社会工作','制药/医疗') then '卫生和社会工作'
             when a.CompanyIndustry in ('文化、体育和娱乐业','广告/媒体') then '文化、体育和娱乐业'
             when a.CompanyIndustry in ('居民服务、修理和其他服务业','服务业') then '居民服务、修理和其他服务业'
             when a.CompanyIndustry in ('房地产业') then '房地产业'
             when a.CompanyIndustry in ('交通运输、仓储和邮政业','物流/运输') then '交通运输、仓储和邮政业'
             when a.CompanyIndustry in ('军队') then '军队'
             when a.CompanyIndustry in ('电力、热力、燃气及水生产和供应业','能源/原材料') then '电力、热力、燃气及水生产和供应业'
             when a.CompanyIndustry in ('水利、环境和公共设施管理业') then '水利、环境和公共设施管理业'
             when a.CompanyIndustry in ('采矿业') then '采矿业'
             when a.CompanyIndustry in ('农、林、牧、渔业') then '农、林、牧、渔业'
             when a.CompanyIndustry in ('国际组织') then '国际组织'
             when a.CompanyIndustry IS NULL then '其他'
             else a.CompanyIndustry end as industry_category,
       type_desc
    FROM (
       SELECT b.SchoolName AS school_name,
              b.SchoolId AS school_id,
              CAST(a.BothMeetId AS VARCHAR) AS fair_id,
              b.Title AS title,
              b.MeetTime AS meet_time,
              a.CompanyName AS company_name,
              a.CompanyIndustry,
              CASE
                 WHEN MONTH(b.MeetTime) > 8 THEN YEAR(b.MeetTime) + 1
                 WHEN MONTH(b.MeetTime) < 9 THEN YEAR(b.MeetTime)
                 ELSE NULL
              END AS graduate_year,
              CASE
                 WHEN b.Type = '0' THEN '双选会'
                 WHEN b.Type = '1' THEN '视频双选会'
                 ELSE NULL
              END AS type_desc
       FROM bus_bothmeet_company a
       LEFT JOIN bus_bothmeet b ON a.BothMeetId = b.Id
       LEFT JOIN sys_org c ON b.SchoolId = c.Id
       WHERE b.IsDeleted = 0
         AND a.AuditStatus != 2
         AND YEAR(b.MeetTime) = YEAR(GETDATE())
         AND c.IsDeleted = 0

       UNION ALL

       SELECT SchoolName AS school_name,
              SchoolId AS school_id,
              CAST(Id AS VARCHAR) AS career_talk_id,
              Title AS title,
              MeetDate AS meet_time,
              a.CompanyName AS company_name,
              a.CompanyIndustry,
              CASE
                 WHEN MONTH(MeetDate) > 8 THEN YEAR(MeetDate) + 1
                 WHEN MONTH(MeetDate) < 9 THEN YEAR(MeetDate)
                 ELSE NULL
              END AS graduate_year,
              '宣讲会' AS type_desc
       FROM bus_career a
       WHERE YEAR(MeetDate) = YEAR(GETDATE())
         AND IsPublish = 1
         AND State != 1
         AND IsDeleted = 0
    ) a
    WHERE graduate_year = {graduate_year};
    '''
    dwhy_xyzp=mssql_data(sql,['type_desc'])#单位行业校园招聘

    datat=pd.crosstab(dwhy_xyzp['industry_category'], dwhy_xyzp['type_desc']).reset_index()
    datat=datat.rename(columns={'industry_category':'行业'})
    datat['总计']=datat.apply(lambda x:x['视频双选会']+x['双选会']+x['宣讲会'],axis=1)
    content1=f"\t 十一、 分行业需求"
    content2=f'''岗位需求量排名靠前的行业分别为{functools.reduce(lambda x,y:x+'、'+y,[f"“{i[0]}”" for i in get_3rd(datat,'总计')])}，分行业校园招聘活动开展情况如下表所示：'''
    add_p(content1,content2)
    add_table(datat)
    print('分行业需求成功！')
except:
    print('分行业需求报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

# 十二、 分行业投递：

In [38]:
try:
    sql=f'''
    select case when a.CompanyIndustry in ('信息传输、软件和信息技术服务业','计算机/互联网/通信/电子','软件和信息技术服务业') then '信息传输、软件和信息技术服务业'
             when a.CompanyIndustry in ('建筑业','房地产/建筑') then '建筑业'
             when a.CompanyIndustry in ('科学研究和技术服务业') then '科学研究和技术服务业'
             when a.CompanyIndustry in ('租赁和商务服务业') then '租赁和商务服务业'
             when a.CompanyIndustry in ('批发和零售业','批发业','零售业') then '批发和零售业'
             when a.CompanyIndustry in ('教育','教育业','专业服务/教育/培训') then '教育'
             when a.CompanyIndustry in ('金融业','会计/金融/银行/保险') then '金融业'
             when a.CompanyIndustry in ('制造业','贸易/消费/制造/营运') then '制造业'
             when a.CompanyIndustry in ('住宿和餐饮业') then '住宿和餐饮业'
             when a.CompanyIndustry in ('公共管理、社会保障和社会组织','政府/非盈利机构/其他') then '公共管理、社会保障和社会组织'
             when a.CompanyIndustry in ('卫生和社会工作','制药/医疗') then '卫生和社会工作'
             when a.CompanyIndustry in ('文化、体育和娱乐业','广告/媒体') then '文化、体育和娱乐业'
             when a.CompanyIndustry in ('居民服务、修理和其他服务业','服务业') then '居民服务、修理和其他服务业'
             when a.CompanyIndustry in ('房地产业') then '房地产业'
             when a.CompanyIndustry in ('交通运输、仓储和邮政业','物流/运输') then '交通运输、仓储和邮政业'
             when a.CompanyIndustry in ('军队') then '军队'
             when a.CompanyIndustry in ('电力、热力、燃气及水生产和供应业','能源/原材料') then '电力、热力、燃气及水生产和供应业'
             when a.CompanyIndustry in ('水利、环境和公共设施管理业') then '水利、环境和公共设施管理业'
             when a.CompanyIndustry in ('采矿业') then '采矿业'
             when a.CompanyIndustry in ('农、林、牧、渔业') then '农、林、牧、渔业'
             when a.CompanyIndustry in ('国际组织') then '国际组织'
             when a.CompanyIndustry IS NULL then '其他'
             else a.CompanyIndustry end as [行业],count(distinct Id) [投递简历数],count(distinct StudentId) [投递人数]
    from(
    select a.Id,a.StudentId,Industry as CompanyIndustry  from bus_job_resume_relation a
    join bus_company_job b on a.JobId=b.Id
    join bus_company c on b.CompanyId=c.Id
    where CASE
            WHEN MONTH(SendTime) > 8 THEN YEAR(SendTime) + 1
            WHEN MONTH(SendTime) < 9 THEN YEAR(SendTime)
            ELSE NULL
        END={graduate_year}
    ) a
    group by case when a.CompanyIndustry in ('信息传输、软件和信息技术服务业','计算机/互联网/通信/电子','软件和信息技术服务业') then '信息传输、软件和信息技术服务业'
             when a.CompanyIndustry in ('建筑业','房地产/建筑') then '建筑业'
             when a.CompanyIndustry in ('科学研究和技术服务业') then '科学研究和技术服务业'
             when a.CompanyIndustry in ('租赁和商务服务业') then '租赁和商务服务业'
             when a.CompanyIndustry in ('批发和零售业','批发业','零售业') then '批发和零售业'
             when a.CompanyIndustry in ('教育','教育业','专业服务/教育/培训') then '教育'
             when a.CompanyIndustry in ('金融业','会计/金融/银行/保险') then '金融业'
             when a.CompanyIndustry in ('制造业','贸易/消费/制造/营运') then '制造业'
             when a.CompanyIndustry in ('住宿和餐饮业') then '住宿和餐饮业'
             when a.CompanyIndustry in ('公共管理、社会保障和社会组织','政府/非盈利机构/其他') then '公共管理、社会保障和社会组织'
             when a.CompanyIndustry in ('卫生和社会工作','制药/医疗') then '卫生和社会工作'
             when a.CompanyIndustry in ('文化、体育和娱乐业','广告/媒体') then '文化、体育和娱乐业'
             when a.CompanyIndustry in ('居民服务、修理和其他服务业','服务业') then '居民服务、修理和其他服务业'
             when a.CompanyIndustry in ('房地产业') then '房地产业'
             when a.CompanyIndustry in ('交通运输、仓储和邮政业','物流/运输') then '交通运输、仓储和邮政业'
             when a.CompanyIndustry in ('军队') then '军队'
             when a.CompanyIndustry in ('电力、热力、燃气及水生产和供应业','能源/原材料') then '电力、热力、燃气及水生产和供应业'
             when a.CompanyIndustry in ('水利、环境和公共设施管理业') then '水利、环境和公共设施管理业'
             when a.CompanyIndustry in ('采矿业') then '采矿业'
             when a.CompanyIndustry in ('农、林、牧、渔业') then '农、林、牧、渔业'
             when a.CompanyIndustry in ('国际组织') then '国际组织'
             when a.CompanyIndustry IS NULL then '其他'
             else a.CompanyIndustry end
                     ORDER BY [投递简历数] desc
    '''
    data_dwhy_jl=mssql_data(sql)
    data_dwhy_jl

    datat=data_dwhy_jl
    content1=f"\t 十二、 分行业投递"
    content2=f'''从毕业生投递的简历来看，求职量排名靠前的行业分别为{functools.reduce(lambda x,y:x+'、'+y,[f"“{i[0]}”" for i in get_3rd(datat,'投递简历数')])}。各行业投递量如下表所示：'''
    add_p(content1,content2)
    add_table(datat,tail=['总计',sum(datat['投递简历数']),sum(datat['投递人数'])])
    print('分行业投递成功！')
except:
    print('分行业投递报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

,行业,投递简历数,投递人数
0,制造业,2539,407
1,建筑业,1565,330
2,信息传输、软件和信息技术服务业,580,192
3,批发和零售业,450,146
4,电力、热力、燃气及水生产和供应业,310,109
5,教育,272,113
6,科学研究和技术服务业,268,122
7,金融业,259,93
8,房地产业,191,85
9,交通运输、仓储和邮政业,170,92


# 十三、 重点企业需求：

In [40]:
try:
    sql=f'''
    select top 10 CompanyName as [公司名称],count(Id) as [发布岗位数]  from bus_company_job
    where CASE
            WHEN MONTH(PublishTime) > 8 THEN YEAR(PublishTime) + 1
            WHEN MONTH(PublishTime) < 9 THEN YEAR(PublishTime)
            ELSE NULL
        END={graduate_year}
    group by CompanyName
    having CompanyName is not null and CompanyName!=''
    ORDER BY [发布岗位数] desc
    '''
    data_zdqy=mssql_data(sql)
    data_zdqy

    datat=data_zdqy
    content1=f"\t 十三、 重点企业需求"
    content2=f'''发布岗位数量排名前10的企业如下表所示：'''
    add_p(content1,content2)
    add_table(datat)
    
    print('重点企业需求成功！')
except:
    print('重点企业需求报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

,公司名称,发布岗位数
0,河北协耀铁路电气化技术有限公司,330
1,北京链家置地房地产经纪有限公司海淀八里庄第一分公司,280
2,北京链家置地房地产经纪有限公司海淀八里庄路分公司,228
3,长沙汉宇房地产经纪有限公司,223
4,深圳市乐有家控股集团有限公司,207
5,北京链家置地房地产经纪有限公司海淀永定路第一分店,164
6,广州高才信息科技有限公司,112
7,人本股份有限公司,96
8,宁波方太厨具有限公司,96
9,浙江京新药业股份有限公司,95


# 导出

In [42]:
try:
    document.save(f'高校毕业生就业数据分析简报.docx')
except:
    print('导出报错，错误信息如下：---------------------------')
    raise
    keyboard.wait('enter')

In [ ]:
print('程序结束，请按enter退出或直接关闭窗口')
keyboard.wait('enter')